# Download CBIS-DDSM on Colab Pro (Full 163GB)
Colab Pro has 200GB space - download entire dataset at once, then split into 4 parts for Kaggle upload

In [ ]:
# Cell 1: Install TCIA Utils
!pip install tcia-utils requests

In [ ]:
# Cell 2: Import libraries and setup
from tcia_utils import nbia
import os
import shutil
import tarfile
from pathlib import Path

# Check available disk space
disk = shutil.disk_usage('/')
print(f"Available disk space: {disk.free / (1024**3):.1f} GB")

# Create download directory
download_dir = '/content/CBIS-DDSM-Full'
os.makedirs(download_dir, exist_ok=True)
print(f"Download directory: {download_dir}")

In [ ]:
# Cell 3: Download FULL 163GB Dataset (Colab Pro has 200GB)
# This will download ALL 6,775 series at once!

collection = "CBIS-DDSM"
print(f"Downloading collection: {collection}")

# Get series data
series_data = nbia.getSeries(collection=collection)
total_series = len(series_data)
print(f"Found {total_series} series to download")

# Extract SeriesInstanceUIDs
series_uids = [item['SeriesInstanceUID'] for item in series_data]

print(f"\n🚀 COLAB PRO: Downloading ALL {total_series} series at once!")
print(f"Estimated size: ~163GB")
print(f"Estimated time: 2-3 hours")
print(f"☕ Go grab lunch/coffee and come back!\n")

# Download everything
import time
start_time = time.time()

nbia.downloadSeries(
    series_data=series_uids,
    input_type="list",
    path=download_dir
)

elapsed = (time.time() - start_time) / 60
print(f"\n✓ Full dataset downloaded in {elapsed:.1f} minutes!")
print(f"Now run Cell 4 to split into 4 parts for Kaggle")

# Check size
!du -sh {download_dir}

In [ ]:
# Cell 4: Split into 4 parts and create archives (FIXED for actual structure)
# Each part will be ~38GB uncompressed, ~28-30GB compressed

import tarfile

print("Splitting dataset into 4 parts...\n")

# Get all series directories (they're directly under base_dir)
base_dir = Path(download_dir)
all_series = [d for d in base_dir.iterdir() if d.is_dir()]

print(f"Found {len(all_series)} series directories")
print(f"Each part will have ~{len(all_series) // 4} series\n")

# Split into 4 equal parts
num_parts = 4
series_per_part = len(all_series) // num_parts

for part_num in range(num_parts):
    start_idx = part_num * series_per_part
    end_idx = start_idx + series_per_part if part_num < num_parts - 1 else len(all_series)
    
    part_series = all_series[start_idx:end_idx]
    part_name = f"CBIS-DDSM-Part{part_num + 1}"
    part_dir = f"/content/{part_name}"
    
    print(f"Creating {part_name} with {len(part_series)} series...")
    os.makedirs(part_dir, exist_ok=True)
    
    # Copy series directories to part directory
    for series_dir in part_series:
        dest_path = Path(part_dir) / series_dir.name
        shutil.copytree(series_dir, dest_path)
    
    # Check uncompressed size
    result = !du -sh {part_dir}
    print(f"  Uncompressed: {result[0].split()[0]}")
    
    # Create compressed archive
    archive_name = f"/content/{part_name}.tar.gz"
    print(f"  Compressing to {part_name}.tar.gz (this takes ~5 mins)...")
    
    with tarfile.open(archive_name, 'w:gz') as tar:
        tar.add(part_dir, arcname=part_name)
    
    archive_size = os.path.getsize(archive_name) / (1024**3)
    print(f"  ✓ Compressed: {archive_size:.1f} GB\n")
    
    # Clean up uncompressed part to save space
    shutil.rmtree(part_dir)

print("✓ All 4 parts created!")
print("\nArchives ready for download:")
!ls -lh /content/*.tar.gz
print("\nFind them in the Files panel (📁) on the left")
print("Download all 4 files, then upload to Kaggle as datasets")

In [ ]:
# Cell 4a: Check actual directory structure first
import os
from pathlib import Path

print("Checking downloaded structure...\n")
base_dir = Path('/content/CBIS-DDSM-Full')

# Show first few levels
for item in list(base_dir.rglob('*'))[:20]:
    print(item)
    
print("\n---")
print(f"Total items: {len(list(base_dir.rglob('*')))}")
print(f"Total .dcm files: {len(list(base_dir.rglob('*.dcm')))}")

In [ ]:
# Cell 5: Download archives to your computer
# These files will appear in the Files panel on the left
# Download each .tar.gz file to your computer

from google.colab import files

print("Download these 4 files:")
print("  1. CBIS-DDSM-Part1.tar.gz")
print("  2. CBIS-DDSM-Part2.tar.gz")
print("  3. CBIS-DDSM-Part3.tar.gz")
print("  4. CBIS-DDSM-Part4.tar.gz")
print("\nYou can find them in the Files panel (📁) on the left")
print("Right-click each file and select 'Download'")
print("\nOR run the cells below to auto-download (slower):")

## Download Instructions

The 4 .tar.gz files are ready in `/content/`

**Option 1: Manual Download (Faster)**
- Find files in Files panel (📁) on the left
- Right-click each file → Download
- Files: CBIS-DDSM-Part1.tar.gz through Part4.tar.gz

**Option 2: Auto-download (Slower)**
- Run cells below to download via browser

In [ ]:
# Cell 7: Auto-download Part 2 (OPTIONAL)
# files.download('/content/CBIS-DDSM-Part2.tar.gz')

In [ ]:
# Cell 8: Auto-download Part 3 (OPTIONAL)
# files.download('/content/CBIS-DDSM-Part3.tar.gz')

In [ ]:
# Cell 9: Auto-download Part 4 (OPTIONAL)
# files.download('/content/CBIS-DDSM-Part4.tar.gz')

## Next Steps:

1. **Download all 4 .tar.gz files** (see above)

2. **Upload to Kaggle as 4 separate datasets:**
   - Go to https://www.kaggle.com/datasets
   - Click "New Dataset"
   - Upload CBIS-DDSM-Part1.tar.gz → Name it "CBIS-DDSM Part 1"
   - Repeat for Parts 2, 3, and 4

3. **Training on Kaggle:**
   - Add all 4 datasets as inputs to your training notebook

   - Extract and combine: `!tar -xzf /kaggle/input/cbis-ddsm-part-X/*.tar.gz`   - Train with 70/30 split using full 163GB dataset!
   - Convert to COCO format